### Лабораторная работа № 6 ###
#### Ансамбли моделей машинного обучения ####
**Цель лабораторной работы:** изучение ансамблей моделей машинного обучения.

Выполнил: Мокренко Никита ИУ5-64

1. Выберите набор данных (датасет) для решения задачи классификации или регресии.
2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.
4. Обучите 1) одну из линейных моделей, 2) SVM и 3) дерево решений. Оцените качество моделей с помощью трех подходящих для задачи метрик. Сравните качество полученных моделей.
5. Произведите для каждой модели подбор одного гиперпараметра с использованием GridSearchCV и кросс-валидации.
6. Повторите пункт 4 для найденных оптимальных значений гиперпараметров. Сравните качество полученных моделей с качеством моделей, полученных в пункте 4.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import *
from sklearn.datasets import load_iris

In [2]:
df = load_iris()
df = pd.DataFrame(data = np.c_[df['data'], df['target']], columns = df['feature_names'] + ['target'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [3]:
df.dtypes

sepal length (cm)    float64
sepal width (cm)     float64
petal length (cm)    float64
petal width (cm)     float64
target               float64
dtype: object

Категориальные признаки отсутствуют

In [4]:
df.isnull().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
target               0
dtype: int64

Пропуски в данных отсутствуют

### 3. Деление выборки на обучающую и тестовую

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df.loc[:, df.columns != 'target'], df['target'],
    test_size=0.2, random_state=1) 

### 4.1. Градиентный бустинг

In [6]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor()
gbr.fit(x_train, y_train)
target1 = gbr.predict(x_test)

In [7]:
print('Средняя абсолютная ошибка: ', mean_absolute_error(y_test, target1), '\n', 
      'Коэффициент детерминации: ',r2_score(y_test, target1), sep = '')

Средняя абсолютная ошибка: 0.04837047215863978
Коэффициент детерминации: 0.9284974462102784


### 4.2. Случайный лес

In [8]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(x_train, y_train)
target2 = rfr.predict(x_test)

/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [9]:
print('Средняя абсолютная ошибка: ', mean_absolute_error(y_test, target2), '\n', 
      'Коэффициент детерминации: ',r2_score(y_test, target2), sep = '')

Средняя абсолютная ошибка: 0.03333333333333333
Коэффициент детерминации: 0.9381443298969072


### 5.1. Подбор параметров для градиентного бустинга

In [10]:
from sklearn.model_selection import GridSearchCV

g_range = np.array(range(10,500,10))

gradient_parameters = [{'n_estimators': g_range} ]

gb_gs = GridSearchCV(GradientBoostingRegressor(), gradient_parameters, cv = 3, scoring = 'r2')
gb_gs.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260,
       270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390,
       400, 410, 420, 430, 440, 450, 460, 470, 480, 490])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [11]:
print('Лучшая модель: ', gb_gs.best_estimator_, '\n', 'Лучшее значение метрики: ', gb_gs.best_score_,
      '\n', 'Лучшее значение параметров: ', gb_gs.best_params_, sep = '')

Лучшая модель: GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=40, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Лучшее значение метрики: 0.9185891713594166
Лучшее значение параметров: {'n_estimators': 40}


### 5.2. Подбор параметров для случайного леса

In [12]:
f_range = np.array(range(1,25,1))

forest_parameters = [{'max_depth': f_range} ]

rf_gs = GridSearchCV(RandomForestRegressor(), forest_parameters, cv = 3, scoring = 'r2')
rf_gs.fit(x_train, y_train)

/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators wi

/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators wi

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [13]:
print('Лучшая модель: ', rf_gs.best_estimator_, '\n', 'Лучшее значение метрики: ', rf_gs.best_score_,
      '\n', 'Лучшее значение параметров: ', rf_gs.best_params_, sep = '')

Лучшая модель: RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=21,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
Лучшее значение метрики: 0.9454502492915591
Лучшее значение параметров: {'max_depth': 21}


### 6.1. Градиентный бустинг

In [14]:
gbr1 = GradientBoostingRegressor(n_estimators = 30)
gbr1.fit(x_train, y_train)
target1_1 = gbr1.predict(x_test)

In [15]:
print('Средняя абсолютная ошибка: ', mean_absolute_error(y_test, target1_1), '\n', 
      'Коэффициент детерминации: ',r2_score(y_test, target1_1), sep = '')

Средняя абсолютная ошибка: 0.07050293194609743
Коэффициент детерминации: 0.9343575845941979


### 6.2. Случайный лес

In [16]:
rfr1 = RandomForestRegressor(max_depth = 5)
rfr1.fit(x_train, y_train)
target2_1 = rfr1.predict(x_test)

/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [17]:
print('Средняя абсолютная ошибка: ', mean_absolute_error(y_test, target2_1), '\n', 
      'Коэффициент детерминации: ',r2_score(y_test, target2_1), sep = '')

Средняя абсолютная ошибка: 0.03333333333333333
Коэффициент детерминации: 0.9381443298969072
